[Bessel's Correction](https://math.oxford.emory.edu/site/math117/besselCorrection/)


# Backprop Ninja


In [ ]:
import matplotlib.pyplot as plt 

import random
random.seed(42)

import torch
import torch.nn.functional as F
torch.manual_seed(42)

import sys
sys.path.insert(0, "..")
from preprocessing import get_data, load_split_data

%matplotlib inline

## Load Data


In [ ]:
block_size = 3
words, stoi, itos = get_data()
Xtr, Ytr, Xdev, Ydev, Xte, Yte = load_split_data(words, stoi, itos)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


## Validate


In [ ]:
def cmp(dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"exact: {ex} | approximate: {app} | maxdiff: {maxdiff}")

## Model


In [ ]:
n_embd = 10
n_hidden = 64
vocab_size = 27

C = torch.randn((vocab_size, n_embd))
# Layer 1
reg = (5 / 3) / ((n_embd * block_size) ** 0.5)
W1 = torch.randn((n_embd * block_size, n_hidden)) * reg
b1 = torch.randn(n_hidden) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size)) * 0.1
b2 = torch.randn(vocab_size) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,))
Xb, Yb = Xtr[ix], Ytr[ix]

4137


## Forward


In [ ]:
emb = C[Xb]  #
embcat = emb.view(emb.shape[0], -1)
# Linear layer 1
hprebn = embcat @ W1 + b1  # n
# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
# note: Bessel's correction (dividing by n-1, not n)
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact)
# Linear layer 2
logits = h @ W2 + b2
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
# if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
    p.grad = None
for t in [
    logprobs,
    probs,
    counts,
    counts_sum,
    counts_sum_inv,
    norm_logits,
    logit_maxes,
    logits,
    h,
    hpreact,
    bnraw,
    bnvar_inv,
    bnvar,
    bndiff2,
    bndiff,
    hprebn,
    bnmeani,
    embcat,
    emb,
]:
    t.retain_grad()
loss.backward()
loss

tensor(3.6917, grad_fn=<NegBackward0>)

## Backprop


### #1

```py
loss = -logprobs[range(n), Yb].mean()
```

dloss/da = -(a + b + c) /3\
dloss/da = -1/3 a - 1/3 b -1/3 c = -1/3


In [ ]:
n = len(logprobs[range(n), Yb])
n

32

In [ ]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1 / 32
cmp(dlogprobs, logprobs)

exact: True | approximate: True | maxdiff: 0.0


### #2

```py
logprobs = probs.log()
```


In [ ]:
dprobs = 1 / probs * dlogprobs
cmp(dprobs, probs)

exact: True | approximate: True | maxdiff: 0.0


### #3

```py
probs = counts * counts_sum_inv
```


In [ ]:
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
cmp(dcounts_sum_inv, counts_sum_inv)

exact: True | approximate: True | maxdiff: 0.0


### #4

```py
counts_sum_inv = counts_sum**-1
```


In [ ]:
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
cmp(dcounts_sum, counts_sum)

exact: True | approximate: True | maxdiff: 0.0


### #5

```py
probs = counts * counts_sum_inv
```


In [ ]:
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
dcounts_ = counts_sum_inv * dprobs
cmp(dcounts_, counts)

exact: False | approximate: False | maxdiff: 0.005659828893840313


### #6

```py
counts_sum = counts.sum(1, keepdims=True)
```


In [ ]:
counts.shape, dcounts_sum.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
dcounts = dcounts_ + 1 * dcounts_sum
cmp(dcounts, counts)

exact: True | approximate: True | maxdiff: 0.0


### #7

```py
counts = norm_logits.exp()
```


In [ ]:
dnorm_logits = norm_logits.exp() * dcounts
cmp(dnorm_logits, norm_logits)

exact: True | approximate: True | maxdiff: 0.0


### #8

```py
norm_logits = logits - logit_maxes
```


In [ ]:
logits.shape, logit_maxes.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
dlogits_ = 1 * dnorm_logits
cmp(dlogits_, norm_logits)

exact: True | approximate: True | maxdiff: 0.0


### #9

```py
norm_logits = logits - logit_maxes
```


In [ ]:
logit_maxes.shape

torch.Size([32, 1])

In [ ]:
dlogit_maxes = (1 * -dnorm_logits).sum(1, keepdims=True)
cmp(dlogit_maxes, logit_maxes)

exact: True | approximate: True | maxdiff: 0.0


### #10

```py
logit_maxes = logits.max(1, keepdim=True).values
```


In [ ]:
logits.max(1, keepdim=True).values.shape

torch.Size([32, 1])

In [ ]:
one_hot = torch.zeros_like(logits)
one_hot[range(32), logits.max(1).indices] = 1
one_hot[0]
# F.one_hot(logits.max(1).indices, num_classes=27)[0]
# plt.imshow(one_hot)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
dlogits = dlogits_ + one_hot * dlogit_maxes
cmp(dlogits, logits)

exact: True | approximate: True | maxdiff: 0.0


### #11

```py
logits = h @ W2 + b2
```


In [ ]:
dlogits.shape, W2.T.shape

(torch.Size([32, 27]), torch.Size([27, 64]))

In [ ]:
dh = dlogits @ W2.T
cmp(dh, h)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
h.T.shape, dlogits.shape

(torch.Size([64, 32]), torch.Size([32, 27]))

In [ ]:
dW2 = h.T @ dlogits
cmp(dW2, W2)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
db2 = dlogits.sum(0)
cmp(db2, b2)

exact: True | approximate: True | maxdiff: 0.0


### #12

```py
h = torch.tanh(hpreact)
```


In [ ]:
dhpreact = (1 - h**2) * dh
cmp(dhpreact, hpreact)

exact: True | approximate: True | maxdiff: 0.0


### #13

```py
hpreact = bngain * bnraw + bnbias
```


In [ ]:
hpreact.shape, bnraw.shape, bngain.shape, bnbias.shape

(torch.Size([32, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([1, 64]))

In [ ]:
dbngain = (bnraw * dhpreact).sum(0)
cmp(dbngain, bngain)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
dbnraw = bngain * dhpreact
cmp(dbnraw, bnraw)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
dbnbias = 1 * dhpreact.sum(0, keepdim=True)
cmp(dbnbias, bnbias)

exact: True | approximate: True | maxdiff: 0.0


# 14

```py
bnraw = bndiff * bnvar_inv
```


In [ ]:
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp(dbnvar_inv, bnvar_inv)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
dbndiff_ = bnvar_inv * dbnraw
cmp(dbndiff_, bndiff)

exact: False | approximate: False | maxdiff: 0.0012920041335746646


### #15

```py
bnvar_inv = (bnvar + 1e-5) ** -0.5
```


In [ ]:
dbnvar = -0.5 * (bnvar + 1e-5) ** -1.5 * dbnvar_inv
cmp(dbnvar, bnvar)

exact: True | approximate: True | maxdiff: 0.0


### 16

```py
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)
```


In [ ]:
dbndiff2 = 1 / (n - 1) * dbnvar
cmp(dbndiff2, bndiff2)

exact: True | approximate: True | maxdiff: 0.0


### #17

```py
bndiff2 = bndiff**2
```


In [ ]:
dbndiff = dbndiff_ + 2 * bndiff * dbndiff2
cmp(dbndiff, bndiff)

exact: True | approximate: True | maxdiff: 0.0


### #18

```py
bndiff = hprebn - bnmeani
```


In [ ]:
dbnmeani = (-1 * dbndiff).sum(0, keepdim=True)
cmp(dbnmeani, bnmeani)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
dhprebn_ = 1 * dbndiff
cmp(dhprebn_, hprebn)

exact: False | approximate: False | maxdiff: 0.0009250685106962919


### #19

```py
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
```


In [ ]:
dhprebn = (1 / n * dbnmeani) + dhprebn_
cmp(dhprebn, hprebn)

exact: True | approximate: True | maxdiff: 0.0


### #20

```py
hprebn = embcat @ W1 + b1
```


In [ ]:
hprebn.shape, embcat.shape, W1.shape, b1.shape

(torch.Size([32, 64]),
 torch.Size([32, 30]),
 torch.Size([30, 64]),
 torch.Size([64]))

In [ ]:
dembcat = dhprebn @ W1.T
cmp(dembcat, embcat)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
dW1 = embcat.T @ dhprebn
cmp(dW1, W1)

exact: True | approximate: True | maxdiff: 0.0


In [ ]:
db1 = dhprebn.sum(0, keepdim=True)
cmp(db1, b1)

exact: True | approximate: True | maxdiff: 0.0


### #21

```py
embcat = emb.view(emb.shape[0], -1)
```


In [ ]:
emb.shape, dembcat.shape

(torch.Size([32, 3, 10]), torch.Size([32, 30]))

In [ ]:
demb = 1 * dembcat.view(emb.shape)
cmp(demb, emb)

exact: True | approximate: True | maxdiff: 0.0


### #22

```py
emb = C[Xb]
```


In [ ]:
C[Xb].shape, C.shape

(torch.Size([32, 3, 10]), torch.Size([27, 10]))

In [ ]:
dC = torch.zeros_like(C)
for i in range(len(Xb)):
    for j in range(Xb.shape[1]):
        ix = Xb[i, j]
        dC[ix] += 1 * demb[i, j]
cmp(dC, C)

exact: True | approximate: True | maxdiff: 0.0
